In [ ]:
# import pandas as pd
# import glob

# # Cari semua file parquet batch
# parquet_files = glob.glob("sepsis_batch_*.parquet")

In [ ]:
# # Baca semua dan gabung
# dfs = [pd.read_parquet(f) for f in parquet_files]
# df_all = pd.concat(dfs, ignore_index=True)

# # Simpan sekali lagi kalau mau jadi satu file
# df_all = df_all.sort_values(by=["stay_id", "hr"])
# df_all.to_parquet("sepsis_full.parquet", index=False)

# print(f"All batches merged! Shape: {df_all.shape}")

All batches merged! Shape: (7200000, 135)


In [ ]:
# conda create -n ml-env python=3.10 -y && conda activate ml-env && conda install -c conda-forge numpy=1.26.4 pandas=2.3.3 scipy=1.15.3 scikit-learn=1.7.2 matplotlib=3.10.6 seaborn=0.13.2 pytorch=2.5.1 torchvision=0.20.1 torchaudio=2.5.1 ipython ipykernel tqdm psutil pyyaml -y && pip install accelerate==1.11.0 aiohappyeyeballs==2.6.1 aiohttp==3.12.15 aiosignal==1.4.0 alembic==1.17.0 annotated-types==0.7.0 anyio==4.11.0 attrs==25.3.0 Authlib==1.6.5 blinker==1.9.0 cachetools==6.2.2 certifi==2025.11.12 charset-normalizer==3.4.4 click==8.3.0 cloudpickle==3.1.2 colorama==0.4.6 colorlog==6.10.1 cryptography==45.0.7 cyclopts==3.24.0 datasets==4.1.1 dill==0.4.0 dnspython==2.8.0 docker==7.1.0 docstring_parser==0.17.0 evaluate==0.4.6 fastapi==0.119.0 Flask==3.1.2 flask-cors==6.0.1 fsspec==2025.9.0 ftfy==6.3.1 GitPython==3.1.45 google-api-core==2.29.0 google-auth==2.41.1 google-cloud-bigquery==3.40.0 google-cloud-core==2.5.0 graphene==3.4.3 grpcio==1.76.0 httpx==0.28.1 huggingface-hub==0.35.1 imbalanced-learn==0.14.0 joblib==1.5.2 kornia==0.8.1 lightning-utilities==0.15.2 mlflow==3.6.0 multiprocess==0.70.16 networkx open_clip_torch==3.2.0 optuna==4.5.0 pillow==12.0.0 protobuf==6.33.4 pydantic==2.12.0 pydantic-settings==2.11.0 pyarrow==21.0.0 regex==2025.9.18 rich==14.2.0 safetensors==0.6.2 SQLAlchemy==2.0.44 starlette==0.48.0 sympy==1.13.1 timm==1.0.20 tokenizers==0.22.1 transformers==4.57.1 uvicorn==0.37.0 waitress==3.0.2 xgboost==3.0.5


In [1]:
import pyarrow.parquet as pq
import pandas as pd

file_path = "/root/sepsis_full2.parquet"

table = pq.read_table(file_path)
df_all = table.to_pandas()

print("Shape:", df_all.shape)
print(df_all.head())

Shape: (2000000, 135)
    row_id  subject_id   stay_id  hr           starttime             endtime  \
0  1808948    12466550  30000153   1 2174-09-29 13:00:00 2174-09-29 14:00:00   
1  1044876    12466550  30000153   4 2174-09-29 16:00:00 2174-09-29 17:00:00   
2  1351780    12466550  30000153   4 2174-09-29 16:00:00 2174-09-29 17:00:00   
3   606495    12466550  30000153   9 2174-09-29 21:00:00 2174-09-29 22:00:00   
4  1645733    12466550  30000153  11 2174-09-29 23:00:00 2174-09-30 00:00:00   

   age height  weight gender  ...  respiration  coagulation  liver  \
0   61   None    73.0      M  ...            0            0      0   
1   61   None    73.0      M  ...            0            0      0   
2   61   None    73.0      M  ...            0            0      0   
3   61   None    73.0      M  ...            0            0      0   
4   61   None    73.0      M  ...            0            0      0   

   cardiovascular  cns  renal  hours_beforesepsis  sepsis  fod  \
0         

In [2]:
# import pandas as pd

# file_path = r"C:\Users\owen\PPNinja\sepsis_batch_1_100000.parquet"
# df_all = pd.read_parquet(file_path)

# print("Shape:", df_all.shape)
# print(df_all.head())

In [3]:
output_cols = [
    "respiration",
    "coagulation",
    "liver",
    "cardiovascular",
    "cns",
    "renal",
    "hours_beforesepsis",
    "sepsis",
    "fod",
    "hours_beforedeath"
]

non_output_cols = df_all.columns.difference(output_cols)
nan_input = df_all[non_output_cols].columns[df_all[non_output_cols].isnull().any()].tolist()

df_imputed = df_all.copy()
df_imputed["hours_beforesepsis"] = df_imputed["hours_beforesepsis"].fillna(0)
df_imputed['gender'] = df_imputed['gender'].map({'M':0, 'F':1})

for col in df_imputed.select_dtypes(include=['object']).columns:
    df_imputed[col] = pd.to_numeric(df_imputed[col], errors='coerce')

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from torch.amp import autocast, GradScaler

In [5]:
X = df_imputed[non_output_cols].drop(columns=["starttime", "endtime", "subject_id"]).to_numpy(dtype=np.float32)
y = df_imputed[output_cols].drop(columns=["fod"]).to_numpy(dtype=np.float32)

In [ ]:
class TemporalWindowDataset(Dataset):
    def __init__(
        self,
        X, y,
        stay_ids,
        times,
        global_feat_mean,   # [F] computed from TRAIN SET ONLY
        window_size=24,
        horizon=1
    ):
        self.X = X
        self.y = y
        self.global_mean = global_feat_mean
        self.samples = []

        df = pd.DataFrame({
            "stay_id": stay_ids,
            "time": times,
            "idx": np.arange(len(stay_ids))
        })

        for stay_id, g in df.groupby("stay_id"):
            g = g.sort_values("time")
            idxs = g["idx"].values
            tvals = g["time"].values

            for i in range(window_size, len(idxs) - horizon):
                hist = idxs[i - window_size:i]
                target = idxs[i + horizon]
                self.samples.append((hist, target, tvals[i - window_size:i]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        hist_idx, target_idx, times = self.samples[idx]

        X_seq = self.X[hist_idx].astype(float)   # [T, F]
        y_target = self.y[target_idx].astype(float)

        # 1️⃣ mask FIRST (raw)
        mask = ~np.isnan(X_seq)

        T, F = X_seq.shape
        X_filled = np.zeros_like(X_seq)
        delta = np.zeros_like(X_seq)

        # 2️⃣ causal GRU-D style fill
        for f in range(F):
            last_val = self.global_mean[f]
            last_time = times[0]

            for t in range(T):
                if mask[t, f]:
                    delta[t, f] = 0.0
                    last_val = X_seq[t, f]
                    last_time = times[t]
                    X_filled[t, f] = last_val
                else:
                    delta[t, f] = times[t] - last_time
                    gamma = np.exp(-delta[t, f])  # simple decay
                    X_filled[t, f] = gamma * last_val + (1 - gamma) * self.global_mean[f]
                    last_val = X_filled[t, f]

        return {
            "X": torch.tensor(X_filled, dtype=torch.float32),
            "mask": torch.tensor(mask.astype(float), dtype=torch.float32),
            "delta": torch.tensor(delta, dtype=torch.float32),
            "y": torch.tensor(y_target, dtype=torch.float32)
        }

def collate_fn(batch):
    X = pad_sequence([b["X"] for b in batch], batch_first=True)
    mask = pad_sequence([b["mask"] for b in batch], batch_first=True)
    delta = pad_sequence([b["delta"] for b in batch], batch_first=True)
    y = torch.stack([b["y"] for b in batch])

    return {"X": X, "mask": mask, "delta": delta, "y": y}

In [ ]:
class TemporalAttnPool(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.score = nn.Linear(d_model, 1)

    def forward(self, z, padding_mask):
        # z: [B, T, D]
        scores = self.score(z).squeeze(-1)  # [B, T]

        # padding_mask: [B, T], 1 = valid, 0 = pad
        scores = scores.masked_fill(padding_mask == 0, -1e9)

        alpha = torch.softmax(scores, dim=1)
        pooled = (z * alpha.unsqueeze(-1)).sum(dim=1)
        return pooled

class GRUDTransformer(nn.Module):
    def __init__(
        self,
        n_features,
        hidden_size=64,
        d_model=128,
        nhead=4,
        num_layers=2,
        out_dim=1
    ):
        super().__init__()

        self.input_size = n_features * 3
        self.gru = nn.GRU(self.input_size, hidden_size, batch_first=True)

        self.to_dmodel = nn.Linear(hidden_size, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        self.attn_pool = TemporalAttnPool(d_model)
        self.head = nn.Linear(d_model, out_dim)

    def forward(self, x, mask, delta):
        inp = torch.cat([x, mask, delta], dim=-1)
        h, _ = self.gru(inp)

        z = self.to_dmodel(h)

        time_mask = (mask.sum(dim=-1) > 0)  # bool [B, T]

        z = self.transformer(
            z,
            src_key_padding_mask=~time_mask
        )

        pooled = self.attn_pool(z, time_mask)
        out = self.head(pooled)
        return out

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print("Supports bf16:", torch.cuda.is_bf16_supported())

Using device: cuda
Supports bf16: True


In [11]:
from sklearn.preprocessing import StandardScaler

n_samples, n_features = X.shape

X_scaled = np.memmap(
    'X_scaled2.dat',
    dtype='float32',
    mode='w+',
    shape=(n_samples, n_features)
)

scaler = StandardScaler()
scaler.fit(X.astype(np.float32))   # FIT SEKALI, GLOBAL

chunk_size = 100_000
for start in range(0, n_samples, chunk_size):
    end = min(start + chunk_size, n_samples)
    X_scaled[start:end] = scaler.transform(
        X[start:end].astype(np.float32)
    )

# target scaling
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.astype(np.float32))

stay_ids = df_imputed['stay_id'].values
times = df_imputed.groupby("stay_id").cumcount().values

global_feat_mean = np.nanmean(X_scaled, axis=0)
global_feat_mean = np.nan_to_num(global_feat_mean, nan=0.0)

dataset = TemporalWindowDataset(
    X=X_scaled,
    y=y_scaled,
    stay_ids=stay_ids,
    times=times,
    global_feat_mean=global_feat_mean,
    window_size=24,
    horizon=1
)

val_ratio = 0.2
n_total = len(dataset)
n_val = int(n_total * val_ratio)
n_train = n_total - n_val

train_dataset, val_dataset = random_split(
    dataset,
    [n_train, n_val],
    generator=torch.Generator().manual_seed(42)
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    collate_fn=collate_fn,
    pin_memory=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=128,
    shuffle=False,
    collate_fn=collate_fn,
    pin_memory=True
)


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device, " | BF16 supported:", torch.cuda.is_bf16_supported())

n_features = X_scaled.shape[1]
hidden_size = 64
output_size = y_scaled.shape[1]

model = GRUDTransformer(
    n_features=X_scaled.shape[1],
    hidden_size=64,
    d_model=128,
    nhead=4,
    num_layers=2,
    out_dim=y_scaled.shape[1]
).to(device)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)

criterion = nn.SmoothL1Loss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3
)

Using device: cuda  | BF16 supported: True


In [ ]:
for epoch in range(10):
    # --- Training ---
    model.train()
    train_losses = []
    train_losses_per_output = []

    loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1} [Train]", leave=False)
    for batch_idx, batch in enumerate(loop):
        X_batch = batch["X"].to(device)
        mask_batch = batch["mask"].to(device)
        delta_batch = batch["delta"].to(device)
        y_batch = batch["y"].to(device)

        optimizer.zero_grad(set_to_none=True)

        with autocast(device_type='cuda', dtype=torch.bfloat16, enabled=use_amp):
            output = model(X_batch, mask_batch, delta_batch)  # [B, output_size]

            # --- per-output loss pake criterion ---
            losses_per_output = torch.stack([
                criterion(output[:, i], y_batch[:, i])
                for i in range(output.shape[1])
            ])
            loss = losses_per_output.mean()

        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_losses_per_output.append(losses_per_output.detach().cpu().numpy())

        if batch_idx % 10 == 0:
            loop.set_postfix({f"L{i}": f"{l:.4f}" for i, l in enumerate(losses_per_output)})

    avg_train_loss = np.mean(train_losses)
    avg_train_losses_per_output = np.mean(train_losses_per_output, axis=0)

    # --- Validation ---
    model.eval()
    val_losses = []
    val_losses_per_output = []

    val_loop = tqdm(val_dataloader, desc=f"Epoch {epoch+1} [Val]", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            X_val = batch["X"].to(device)
            mask_val = batch["mask"].to(device)
            delta_val = batch["delta"].to(device)
            y_val = batch["y"].to(device)

            with autocast(device_type='cuda', dtype=torch.bfloat16, enabled=use_amp):
                val_out = model(X_val, mask_val, delta_val)

                val_losses_batch = torch.stack([
                    criterion(val_out[:, i], y_val[:, i])
                    for i in range(val_out.shape[1])
                ])
                val_loss = val_losses_batch.mean()

            val_losses.append(val_loss.item())
            val_losses_per_output.append(val_losses_batch.cpu().numpy())

    avg_val_loss = np.mean(val_losses)
    avg_val_losses_per_output = np.mean(val_losses_per_output, axis=0)

    scheduler.step(avg_val_loss)

    print(
        f"Epoch {epoch+1} | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Train per output: {avg_train_losses_per_output} | "
        f"Val Loss: {avg_val_loss:.4f} | "
        f"Val per output: {avg_val_losses_per_output}"
    )

Epoch 1 | Train Loss: 0.1908 | Train per output: [0.18716806 0.11450574 0.13004689 0.10756511 0.25459883 0.14624625
 0.08314896 0.44984046 0.24423419] | Val Loss: 0.1869 | Val per output: [0.1842378  0.11657601 0.12781377 0.10975344 0.24924178 0.14479917
 0.08031132 0.43886927 0.23059319]


Epoch 2 | Train Loss: 0.1794 | Train per output: [0.17966814 0.11418217 0.12582032 0.1078416  0.2513497  0.14399204
 0.07467324 0.39882758 0.21844211] | Val Loss: 0.1683 | Val per output: [0.1762729  0.11112966 0.11888427 0.10718183 0.24541065 0.13978374
 0.06015815 0.35980946 0.19566399]


Epoch 3 [Val]:  23%|██▎       | 268/1175 [01:02<05:34,  2.71it/s]                                                                                                      